# Curate kg.csv

In [1]:
import pandas as pd

df = pd.read_csv("D:/Lab/Research/EMERGE-REPLICATE/datasets/dataverse_files/kg.csv", low_memory=False)

df = df[["relation", "x_index", "y_index"]]

df.head()

,relation,x_index,y_index
0,protein_protein,0,8889
1,protein_protein,1,2798
2,protein_protein,2,5646
3,protein_protein,3,11592
4,protein_protein,4,2122


In [2]:
from collections import defaultdict
import pickle
from tqdm import tqdm

adj_list = defaultdict(list)
for u, v, r in tqdm(zip(df["x_index"].values, df["y_index"].values, df["relation"].values), total=len(df)):
    adj_list[int(u)].append((int(v), str(r)))

out_path = "D:/Lab/Research/EMERGE-REPLICATE/rag/curated_data/kg_adjacency.pkl"
with open(out_path, "wb") as f:
    pickle.dump(adj_list, f, protocol=pickle.HIGHEST_PROTOCOL)


100%|██████████| 8100498/8100498 [00:03<00:00, 2082465.48it/s]


# Curate disease_features.csv

In [3]:
df = pd.read_csv("D:/Lab/Research/EMERGE-REPLICATE/datasets/dataverse_files/disease_features.csv", low_memory=False)
df = df.sort_values("node_index").reset_index(drop=True)
df = df[["node_index", "mondo_name", "mondo_definition", "orphanet_definition", "umls_description"]]
df.head()

,node_index,mondo_name,mondo_definition,orphanet_definition,umls_description
0,27158,osteogenesis imperfecta type 5,Osteogenesis imperfecta type V is a moderate t...,A moderate form of osteogenesis imperfecta cha...,Osteogenesis imperfecta is a genetic disorder ...
1,27158,osteogenesis imperfecta type 5,Osteogenesis imperfecta type V is a moderate t...,A moderate form of osteogenesis imperfecta cha...,A group of usually autosomal dominant inherite...
2,27158,osteogenesis imperfecta type 7,Any osteogenesis imperfecta in which the cause...,NaN,"collagen diseases characterized by brittle, os..."
3,27158,osteogenesis imperfecta,Osteogenesis imperfecta (OI) comprises a heter...,Osteogenesis imperfecta (OI) comprises a heter...,A group of inherited connective tissue disorde...
4,27158,osteogenesis imperfecta,Osteogenesis imperfecta (OI) comprises a heter...,Osteogenesis imperfecta (OI) comprises a heter...,Congenital structural abnormalities and deform...


In [ ]:
from ..utils.bgem3 import batch_encode

texts = (
    df["mondo_name"].fillna("") + " " +
    df["mondo_definition"].fillna("") + " " +
    df["orphanet_definition"].fillna("") + " " +
    df["umls_description"].fillna("")
).tolist()

df["embed"] = list(batch_encode(texts, batch_size=64, max_length=8192).cpu().numpy())
df.head()

d:\Lab\Research\EMERGE-REPLICATE\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pre tokenize: 100%|██████████| 690/690 [00:02<00:00, 292.60it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 690/690 [03:04<00:00,  3.74it/s]


,node_index,mondo_name,mondo_definition,orphanet_definition,umls_description,embed
0,27158,osteogenesis imperfecta type 5,Osteogenesis imperfecta type V is a moderate t...,A moderate form of osteogenesis imperfecta cha...,Osteogenesis imperfecta is a genetic disorder ...,"[0.00514, -0.002495, -0.0543, 0.002737, -0.014..."
1,27158,osteogenesis imperfecta type 5,Osteogenesis imperfecta type V is a moderate t...,A moderate form of osteogenesis imperfecta cha...,A group of usually autosomal dominant inherite...,"[0.00846, -0.013336, -0.03964, 0.01071, -0.017..."
2,27158,osteogenesis imperfecta type 7,Any osteogenesis imperfecta in which the cause...,NaN,"collagen diseases characterized by brittle, os...","[-0.01063, 0.03102, -0.02197, -0.0166, -0.0324..."
3,27158,osteogenesis imperfecta,Osteogenesis imperfecta (OI) comprises a heter...,Osteogenesis imperfecta (OI) comprises a heter...,A group of inherited connective tissue disorde...,"[-0.01525, 0.02022, -0.06186, 0.0002285, -0.03..."
4,27158,osteogenesis imperfecta,Osteogenesis imperfecta (OI) comprises a heter...,Osteogenesis imperfecta (OI) comprises a heter...,Congenital structural abnormalities and deform...,"[-0.01671, 0.01733, -0.05927, 0.000125, -0.033..."


In [5]:
out_path = "D:/Lab/Research/EMERGE-REPLICATE/rag/curated_data/disease_features_cleaned.pkl"
with open(out_path, "wb") as f:
    pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)